In [49]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import GridSearchCV

userfiles = os.listdir('data_normalized/fold_1/train')

for user in userfiles:
    for fold in [1,2,3,4]:
        for interval in [2,3,4]:
            train_data = pd.read_csv('interval_datasets_normalized/window_{}/fold_{}/train/{}'.format(interval, fold, user))
            test_data = pd.read_csv('interval_datasets_normalized/window_{}/fold_{}/test/{}'.format(interval, fold, user))

            train_days = np.array((train_data['predict_day'])).reshape(-1)
            test_days = np.array((test_data['predict_day'])).reshape(-1)

            trainY = np.array(train_data['true_mood'])
            train_data = train_data.drop(['predict_day', 'user', 'window', 'true_mood'], axis=1)

            testY = np.array(test_data['true_mood'])
            test_data = test_data.drop(['predict_day', 'user', 'window', 'true_mood'], axis=1)

            trainY = trainY.reshape(-1)
            testY = testY.reshape(-1)

            trainX = np.array(train_data)
            testX = np.array(test_data)

            tol_list = [1e-4, 1e-3, 1e-2, 1e-1]
            C_range = [0.1, 0.5, 1, 1.5]
            eps_range = [0.01, 0.1, 0.5]
            kernel_list = ['linear', 'rbf', 'sigmoid', 'poly']

            param_grid = dict(C=C_range, epsilon=eps_range, kernel=kernel_list, tol=tol_list)
            grid = GridSearchCV(svm.SVR(), param_grid, cv=5, scoring='neg_mean_squared_error')
            grid.fit(trainX, trainY)

            bestparams = grid.best_params_

            C = bestparams['C']
            epsilon = bestparams['epsilon']
            tol = bestparams['tol']
            kernel = bestparams['kernel']

            clf = svm.SVR(C=C, epsilon=epsilon, kernel=kernel, tol=tol)
            clf.fit(trainX, trainY) 
            test_predictions = clf.predict(testX)
            train_predictions = clf.predict(trainX)

            test_errors = testY - test_predictions
            train_errors = trainY - train_predictions
            parameters = [C, epsilon, tol, kernel]

            pd.DataFrame([train_days, train_errors, test_days, test_errors, bestparams.keys(), bestparams.values()])\
                .to_csv('data_normalized_res/svm_noweekdays/fold_{}/{}'.format(fold, user))

            print user, fold, interval

AS14.01.csv 1 2
AS14.01.csv 1 3
AS14.01.csv 1 4
AS14.01.csv 2 2
AS14.01.csv 2 3
AS14.01.csv 2 4
AS14.01.csv 3 2
AS14.01.csv 3 3
AS14.01.csv 3 4
AS14.01.csv 4 2
AS14.01.csv 4 3
AS14.01.csv 4 4
AS14.02.csv 1 2
AS14.02.csv 1 3
AS14.02.csv 1 4
AS14.02.csv 2 2
AS14.02.csv 2 3
AS14.02.csv 2 4
AS14.02.csv 3 2
AS14.02.csv 3 3
AS14.02.csv 3 4
AS14.02.csv 4 2
AS14.02.csv 4 3
AS14.02.csv 4 4
AS14.03.csv 1 2
AS14.03.csv 1 3
AS14.03.csv 1 4
AS14.03.csv 2 2
AS14.03.csv 2 3
AS14.03.csv 2 4
AS14.03.csv 3 2
AS14.03.csv 3 3
AS14.03.csv 3 4
AS14.03.csv 4 2
AS14.03.csv 4 3
AS14.03.csv 4 4
AS14.05.csv 1 2
AS14.05.csv 1 3
AS14.05.csv 1 4
AS14.05.csv 2 2
AS14.05.csv 2 3
AS14.05.csv 2 4
AS14.05.csv 3 2
AS14.05.csv 3 3
AS14.05.csv 3 4
AS14.05.csv 4 2
AS14.05.csv 4 3
AS14.05.csv 4 4
AS14.06.csv 1 2
AS14.06.csv 1 3
AS14.06.csv 1 4
AS14.06.csv 2 2
AS14.06.csv 2 3
AS14.06.csv 2 4
AS14.06.csv 3 2
AS14.06.csv 3 3
AS14.06.csv 3 4
AS14.06.csv 4 2
AS14.06.csv 4 3
AS14.06.csv 4 4
AS14.07.csv 1 2
AS14.07.csv 1 3
AS14.07.